In [1]:
# Diagnostic Multi-Watermarking Experiment
from audioseal import AudioSeal
import torch
import torchaudio
import numpy as np

# Load models
generator = AudioSeal.load_generator("audioseal_wm_16bits")
detector = AudioSeal.load_detector("audioseal_detector_16bits")

# Download original audio (reusing previous download method)
audio, sr = torchaudio.load("test.wav")
audios = audio.unsqueeze(0)  # Add batch dimension

def debug_watermarking():
    # Detailed debugging of watermarking process
    print("Watermarking Debugging:")
    
    # Generate first watermark
    watermark = generator.get_watermark(audios, sample_rate=sr)
    
    # Debug watermark characteristics
    print("\nWatermark Characteristics:")
    print(f"Watermark Shape: {watermark.shape}")
    print(f"Watermark Mean: {watermark.mean().item()}")
    print(f"Watermark Std: {watermark.std().item()}")
    
    # Original audio characteristics
    print("\nOriginal Audio Characteristics:")
    print(f"Audio Shape: {audios.shape}")
    print(f"Audio Mean: {audios.mean().item()}")
    print(f"Audio Std: {audios.std().item()}")
    
    # Watermarked audio
    watermarked_audio = audios + watermark
    
    # Calculate SNR manually
    noise = watermarked_audio - audios
    manual_snr = 10 * torch.log10(audios.pow(2).mean() / noise.pow(2).mean())
    print(f"\nManual SNR Calculation: {manual_snr.item():.2f} dB")
    
    # Watermark detection
    message = torch.randint(0, 2, (1, 16), dtype=torch.int32)
    detector.message = message
    
    result, detected_msg = detector.detect_watermark(
        watermarked_audio, 
        sample_rate=sr, 
        message_threshold=0.5
    )
    
    print("\nWatermark Detection:")
    print(f"Original Message:  {message.numpy().flatten()}")
    print(f"Detected Message:  {detected_msg.round().numpy().flatten()}")
    print(f"Detection Probability: {result:.4f}")
    
    # Bit Error Rate
    ber = (message != detected_msg.round()).float().mean()
    print(f"Bit Error Rate: {ber:.4f}")

# Run diagnostic function
debug_watermarking()

# Additional diagnostic for multiple watermarks
def multi_watermark_diagnostic():
    print("\n--- Multi-Watermark Diagnostic ---")
    
    # Prepare audio
    working_audio = audios.clone()
    
    # Generate multiple messages
    messages = [torch.randint(0, 2, (1, 16), dtype=torch.int32) for _ in range(4)]
    
    for i, msg in enumerate(messages, 1):
        print(f"\nWatermark {i} Embedding:")
        
        # Generate watermark
        watermark = generator.get_watermark(working_audio, sample_rate=sr, message=msg)
        
        # Apply watermark with small strength
        working_audio += 0.1 * watermark
        
        # Detect all previous watermarks
        for j, prev_msg in enumerate(messages[:i], 1):
            detector.message = prev_msg
            result, detected_msg = detector.detect_watermark(
                working_audio, 
                sample_rate=sr, 
                message_threshold=0.5
            )
            
            print(f"  Detecting Watermark {j}:")
            print(f"    Original Message:  {prev_msg.numpy().flatten()}")
            print(f"    Detected Message:  {detected_msg.round().numpy().flatten()}")
            print(f"    Detection Probability: {result:.4f}")
            
            # Bit Error Rate
            ber = (prev_msg != detected_msg.round()).float().mean()
            print(f"    Bit Error Rate: {ber:.4f}")

# Run multi-watermark diagnostic
multi_watermark_diagnostic()

Watermarking Debugging:

Watermark Characteristics:
Watermark Shape: torch.Size([1, 1, 167226])
Watermark Mean: -0.000180509639903903
Watermark Std: 0.00244920258410275

Original Audio Characteristics:
Audio Shape: torch.Size([1, 1, 167226])
Audio Mean: 1.2320103905949509e-06
Audio Std: 0.06579816341400146

Manual SNR Calculation: 28.56 dB

Watermark Detection:
Original Message:  [1 1 1 0 0 1 0 1 1 0 1 0 1 1 1 0]
Detected Message:  [0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 0]
Detection Probability: 1.0000
Bit Error Rate: 0.5000

--- Multi-Watermark Diagnostic ---

Watermark 1 Embedding:
  Detecting Watermark 1:
    Original Message:  [0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1]
    Detected Message:  [0 1 0 0 1 1 1 1 1 0 1 0 0 1 0 1]
    Detection Probability: 0.0001
    Bit Error Rate: 0.5000

Watermark 2 Embedding:
  Detecting Watermark 1:
    Original Message:  [0 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1]
    Detected Message:  [0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1]
    Detection Probability: 0.0002
    Bit Error Rate: 